# Analisi di 28 milioni recensioni di film

In questo notebook utilizzerò Spark con il modulo SparkSQL ed un Dataframe per analizzare oltre 28 milioni di recensioni di film. Nello specifico le domande alla quale cercherò di dare una risposta sono le seguenti:



* **Quante recensioni ci sono esattamente nel dataset ?**

* **Qual è il numero di recensioni medie per utente ?**

* **Quale utente ha scritto più recensioni ? Quante sono le recensioni che ha scritto ? Qual è il suo voto medio ?**

* **Quali sono i 10 film che hanno ricevuto più recensioni ?**

* **Quali sono i 10 film con le recensioni più positive ?**

* **Quali sono i 10 film con le recensioni più negative ?**

* **Quali sono le 10 recensioni più recenti ?**

* **Qual è il film più visto per anno ?**

* **Aggiungiamo titolo e genere alla lista dei film più visti per anno**

Scarico l'archivio dei film sulla mia macchina

In [ ]:
!wget -q http://files.grouplens.org/datasets/movielens/ml-latest.zip

I file che mi interessano sono:

+ **ratings.csv**: che contiene, per ogni riga, id dell'utente, id del film, valutazione da 1.0 a 5.0 e timestamp.
* **movies.csv**: che contiene nome e genere dei film associati agli id.

### Librerie

In [1]:
import os
import pandas as pd

In [2]:
import pyspark.sql.functions
from pyspark.sql.types import *

### Inizializzazione di Spark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()
from pyspark.sql.types import *

### Caricamento del dataset

In [4]:
ratings = spark.read.csv('ml-latest/ratings.csv',header=True,inferSchema=True)

In [5]:
ratings.show(n=5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    307|   3.5|1256677221|
|     1|    481|   3.5|1256677456|
|     1|   1091|   1.5|1256677471|
|     1|   1257|   4.5|1256677460|
|     1|   1449|   4.5|1256677264|
+------+-------+------+----------+
only showing top 5 rows



In [6]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



Andiamo a modificare lo schema.

In [7]:
data_schema = [StructField('userId',StringType(),nullable = True),
               StructField('movieId',StringType(),nullable = True),
               StructField('rating',FloatType(),nullable = True),
               StructField('timestamp',IntegerType(),nullable = True)]

In [8]:
schema = StructType(fields = data_schema)

In [9]:
ratings = spark.read.csv('ml-latest/ratings.csv',header=True,schema = schema)

In [10]:
ratings.show(n=5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    307|   3.5|1256677221|
|     1|    481|   3.5|1256677456|
|     1|   1091|   1.5|1256677471|
|     1|   1257|   4.5|1256677460|
|     1|   1449|   4.5|1256677264|
+------+-------+------+----------+
only showing top 5 rows



In [11]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: integer (nullable = true)



Non rimane che cambiare la colonna Timestamp dal formato unix a uno comprensibile.

In [12]:
from pyspark.sql.functions import from_unixtime

ratings = ratings.withColumn('timestamp',from_unixtime('timestamp', format='yyyy-MM-dd HH:mm:ss'))

In [13]:
ratings.show(n=5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|    307|   3.5|2009-10-27 22:00:21|
|     1|    481|   3.5|2009-10-27 22:04:16|
|     1|   1091|   1.5|2009-10-27 22:04:31|
|     1|   1257|   4.5|2009-10-27 22:04:20|
|     1|   1449|   4.5|2009-10-27 22:01:04|
+------+-------+------+-------------------+
only showing top 5 rows



In [14]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: string (nullable = true)



A questo punto convertiamo il type dell'ultima colonna da stringa a timestamp.

In [15]:
from pyspark.sql.functions import to_utc_timestamp

ratings = ratings.withColumn('timestamp',to_utc_timestamp(ratings['timestamp'],'UTC'))

In [16]:
ratings.show(n=5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|    307|   3.5|2009-10-27 22:00:21|
|     1|    481|   3.5|2009-10-27 22:04:16|
|     1|   1091|   1.5|2009-10-27 22:04:31|
|     1|   1257|   4.5|2009-10-27 22:04:20|
|     1|   1449|   4.5|2009-10-27 22:01:04|
+------+-------+------+-------------------+
only showing top 5 rows



In [17]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: timestamp (nullable = true)



#### Quante recensioni ci sono esattamente nel dataset ?

Contiamo il numero totale di recensioni.

In [18]:
num_recensioni = ratings.count()

In [19]:
print('Il numero di recensioni è',num_recensioni)

Il numero di recensioni è 27753444


#### Qual è il numero di recensioni medie per utente ?

In [20]:
ratings_by_user = ratings.groupBy('userId')

Di seguito il numero di recensioni per utente.

In [21]:
ratings_by_user_count = ratings_by_user.count()

In [22]:
ratings_by_user_count.show()

+------+-----+
|userId|count|
+------+-----+
|   296|   10|
|   467|   10|
|   675|    4|
|   691|   15|
|   829|   15|
|  1090|   12|
|  1159|   29|
|  1436|  100|
|  1512|   79|
|  1572|  106|
|  2069|  854|
|  2088|    1|
|  2136|   10|
|  2162|   28|
|  2294| 2762|
|  2904|    1|
|  3210|   79|
|  3414|   55|
|  3606|   22|
|  3959|  107|
+------+-----+
only showing top 20 rows



Ora vediamo quanti sono i recensori.

In [23]:
num_reviewers = ratings_by_user_count.count()

In [24]:
num_reviewers

283228

Vediamo il numero medio di recensioni per utente.

In [25]:
print('Il numero medio di recensioni per utente è: ',num_recensioni/num_reviewers)

Il numero medio di recensioni per utente è:  97.98976089934611


#### Quale utente ha scritto più recensioni ? Quante sono le recensioni che ha scritto ? Qual è il suo voto medio ?

In [26]:
top_reviewer = ratings_by_user_count.sort('count',ascending = False).first()

In [27]:
top_reviewer

Row(userId='123100', count=23715)

In [28]:
print("L'utente "+str(top_reviewer[0])+" è quello che ha scritto più recensioni. Ne ha scritte "+str(top_reviewer[1]))

L'utente 123100 è quello che ha scritto più recensioni. Ne ha scritte 23715


In [29]:
top_viewer_mean = ratings.filter(ratings.userId == str(top_reviewer[0])).agg({'rating':'mean'}).collect()

In [30]:
print("La valutazione media del recensore più attivo è "+ str(top_viewer_mean[0][0]) + " stelle")

La valutazione media del recensore più attivo è 3.1306346194391734 stelle


#### Quali sono i 10 film che hanno ricevuto più recensioni ?

Di seguito i 10 film col maggior numero di recensioni (in ordine decrescente).

In [31]:
ratings.groupBy("movieId").count().sort('count',ascending = False).show(10)

+-------+-----+
|movieId|count|
+-------+-----+
|    318|97999|
|    356|97040|
|    296|92406|
|    593|87899|
|   2571|84545|
|    260|81815|
|    480|76451|
|    527|71516|
|    110|68803|
|      1|68469|
+-------+-----+
only showing top 10 rows



#### Quali sono i 10 film con le recensioni più positive ? Quali sono i 10 film con le recensioni più negative ?

Vado ad applicarlo sui film con più di 100 recensioni.

10 film con più recensioni positive.

In [32]:
ratings \
.groupBy('movieId') \
.agg({'rating':'mean','movieId':'count'}) \
.where("count(movieId) > 100") \
.sort('avg(rating)', ascending =False).show(10)

+-------+------------------+--------------+
|movieId|       avg(rating)|count(movieId)|
+-------+------------------+--------------+
| 171011|4.4865181711606095|           853|
| 159817| 4.458092485549133|          1384|
|    318| 4.424188001918387|         97999|
| 170705| 4.399898373983739|           984|
| 174053| 4.350558659217877|          1074|
| 171495| 4.343949044585988|           157|
| 172591| 4.339667458432304|           421|
|    858| 4.332892749244713|         60904|
|     50| 4.291958829205532|         62180|
| 176601| 4.263888888888889|           180|
+-------+------------------+--------------+
only showing top 10 rows



10 film con più recensioni negative.

In [33]:
ratings \
.groupBy('movieId') \
.agg({'rating':'mean','movieId':'count'}) \
.where("count(movieId) > 100") \
.sort('avg(rating)', ascending =True).show(10)

+-------+------------------+--------------+
|movieId|       avg(rating)|count(movieId)|
+-------+------------------+--------------+
|   8859|0.8739495798319328|           238|
|   6483|1.0138592750533049|           469|
|   4775| 1.141025641025641|           741|
|   1826|1.2038288288288288|           444|
|   6587|1.2055555555555555|           810|
|  31698|1.2441176470588236|           680|
|   5739|1.2612359550561798|           178|
|  61348|1.2672849915682969|           593|
|   5738|1.3549382716049383|           162|
|   3574|1.3580645161290323|           155|
+-------+------------------+--------------+
only showing top 10 rows



#### Quali sono le 10 recensioni più recenti ?

In [34]:
ratings.sort('timestamp',ascending=False).show(10)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
| 82922| 167780|   4.0|2018-09-26 08:59:09|
| 82922|  53519|   4.0|2018-09-26 08:58:50|
|280481|    494|   3.0|2018-09-26 08:58:47|
|280481|   2355|   3.0|2018-09-26 08:58:43|
|280481|   2294|   2.0|2018-09-26 08:58:41|
|280481| 176101|   3.5|2018-09-26 08:58:30|
|280481|  64614|   3.0|2018-09-26 08:58:22|
| 82922| 165831|   4.0|2018-09-26 08:58:09|
|280481|   1079|   2.5|2018-09-26 08:58:06|
| 82922|  52281|   4.0|2018-09-26 08:58:05|
+------+-------+------+-------------------+
only showing top 10 rows



#### Qual è il film più visto per anno ?

Il film più visto per anno supponiamo sia il film col maggior numero di valutazioni nell'arco dell'anno stesso.

Aggiungo una colonna con solamente l'anno.

In [35]:
from pyspark.sql.functions import year, count,max

In [36]:
ratings_with_year = ratings.withColumn('year',year(ratings['timestamp'])).drop('timestamp')

In [37]:
ratings_with_year.show()

+------+-------+------+----+
|userId|movieId|rating|year|
+------+-------+------+----+
|     1|    307|   3.5|2009|
|     1|    481|   3.5|2009|
|     1|   1091|   1.5|2009|
|     1|   1257|   4.5|2009|
|     1|   1449|   4.5|2009|
|     1|   1590|   2.5|2009|
|     1|   1591|   1.5|2009|
|     1|   2134|   4.5|2009|
|     1|   2478|   4.0|2009|
|     1|   2840|   3.0|2009|
|     1|   2986|   2.5|2009|
|     1|   3020|   4.0|2009|
|     1|   3424|   4.5|2009|
|     1|   3698|   3.5|2009|
|     1|   3826|   2.0|2009|
|     1|   3893|   3.5|2009|
|     2|    170|   3.5|2007|
|     2|    849|   3.5|2007|
|     2|   1186|   3.5|2007|
|     2|   1235|   3.0|2007|
+------+-------+------+----+
only showing top 20 rows



Ora vado a raggruppare il dataframe per anno e film.

In [38]:
ratings_with_year = ratings_with_year.groupBy(['year','movieId']).agg(count('rating').alias('num_reviews'))

In [39]:
ratings_with_year.show()

+----+-------+-----------+
|year|movieId|num_reviews|
+----+-------+-----------+
|2005|    255|         43|
|2005|   1917|       3460|
|2005|   3793|       4089|
|2005|   5064|        788|
|2005|   6966|        257|
|2005|   8912|        155|
|1997|   1057|        585|
|2000|   1094|       2563|
|2015|  68954|       5530|
|2015|  78499|       3050|
|2015|  95167|        996|
|2013|   6378|        327|
|1996|    348|       3364|
|2001|   4066|        398|
|2004|   2409|        334|
|2000|   2671|       1842|
|1999|    293|       1192|
|1999|   1358|       2182|
|1999|   2336|       1446|
|2003|   2406|        697|
+----+-------+-----------+
only showing top 20 rows



In [40]:
from pyspark.sql import Window

In [41]:
windows = Window.partitionBy('year')

In [42]:
ratings_with_year = ratings_with_year.withColumn('max_num_reviews',max('num_reviews').over(windows))

In [43]:
ratings_with_year = ratings_with_year.where(ratings_with_year['num_reviews'] == ratings_with_year['max_num_reviews'])

In [44]:
ratings_with_year.sort('year',ascending = False).show(30)

+----+-------+-----------+---------------+
|year|movieId|num_reviews|max_num_reviews|
+----+-------+-----------+---------------+
|2018|    318|       4311|           4311|
|2017|    318|       7874|           7874|
|2016|    318|       8976|           8976|
|2015|   2571|      12775|          12775|
|2014|    318|       2673|           2673|
|2013|    318|       2713|           2713|
|2012|  79132|       2422|           2422|
|2011|  79132|       3282|           3282|
|2010|  72998|       3884|           3884|
|2009|  58559|       3720|           3720|
|2008|   2571|       4208|           4208|
|2007|   2571|       3409|           3409|
|2006|   7153|       4000|           4000|
|2005|   5952|       6230|           6230|
|2004|   7153|       3699|           3699|
|2003|   5952|       3684|           3684|
|2002|   4993|       3553|           3553|
|2001|   1210|       4516|           4516|
|2000|   1210|       7453|           7453|
|1999|   2396|       4621|           4621|
|1998|   17

Rimuoviamo la colonna "num_reviews" e le righe relative al 1995.

In [45]:
ratings_with_year = ratings_with_year.drop('num_reviews').filter("year != 1995").sort('year',ascending = False)

In [46]:
ratings_with_year.show(30)

+----+-------+---------------+
|year|movieId|max_num_reviews|
+----+-------+---------------+
|2018|    318|           4311|
|2017|    318|           7874|
|2016|    318|           8976|
|2015|   2571|          12775|
|2014|    318|           2673|
|2013|    318|           2713|
|2012|  79132|           2422|
|2011|  79132|           3282|
|2010|  72998|           3884|
|2009|  58559|           3720|
|2008|   2571|           4208|
|2007|   2571|           3409|
|2006|   7153|           4000|
|2005|   5952|           6230|
|2004|   7153|           3699|
|2003|   5952|           3684|
|2002|   4993|           3553|
|2001|   1210|           4516|
|2000|   1210|           7453|
|1999|   2396|           4621|
|1998|   1721|           2398|
|1997|    780|          11350|
|1996|    592|          25760|
+----+-------+---------------+



#### Aggiungiamo titolo e genere alla lista dei film più visti per anno

Per farlo devo importare un altro dataset.

In [47]:
movies = spark.read.csv('ml-latest/movies.csv',header = True, inferSchema=True)

In [48]:
movies.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [49]:
ratings_with_year.show(5)

+----+-------+---------------+
|year|movieId|max_num_reviews|
+----+-------+---------------+
|2018|    318|           4311|
|2017|    318|           7874|
|2016|    318|           8976|
|2015|   2571|          12775|
|2014|    318|           2673|
+----+-------+---------------+
only showing top 5 rows



In [50]:
movies.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [51]:
ratings_with_year.printSchema()

root
 |-- year: integer (nullable = true)
 |-- movieId: string (nullable = true)
 |-- max_num_reviews: long (nullable = true)



A questo punto andiamo a fare semplicemente una join.

In [52]:
left_join = ratings_with_year.join(movies, ratings_with_year.movieId == movies.movieId,how='left')

In [53]:
left_join.select(['year','title','genres','max_num_reviews']).sort('year',ascending=False).show(30)

+----+--------------------+--------------------+---------------+
|year|               title|              genres|max_num_reviews|
+----+--------------------+--------------------+---------------+
|2018|Shawshank Redempt...|         Crime|Drama|           4311|
|2017|Shawshank Redempt...|         Crime|Drama|           7874|
|2016|Shawshank Redempt...|         Crime|Drama|           8976|
|2015|  Matrix, The (1999)|Action|Sci-Fi|Thr...|          12775|
|2014|Shawshank Redempt...|         Crime|Drama|           2673|
|2013|Shawshank Redempt...|         Crime|Drama|           2713|
|2012|    Inception (2010)|Action|Crime|Dram...|           2422|
|2011|    Inception (2010)|Action|Crime|Dram...|           3282|
|2010|       Avatar (2009)|Action|Adventure|...|           3884|
|2009|Dark Knight, The ...|Action|Crime|Dram...|           3720|
|2008|  Matrix, The (1999)|Action|Sci-Fi|Thr...|           4208|
|2007|  Matrix, The (1999)|Action|Sci-Fi|Thr...|           3409|
|2006|Lord of the Rings..